INPUT ALL VARIABLES

In [9]:
WORKDIR=$PWD/UAS_Bioinformatics_Annisa
REF_GENOME_DIR="${WORKDIR}/Reference_Genome"
FASTQ_DIR="${WORKDIR}/Fastq_Data"
TRIMMED_DIR="${WORKDIR}/Trimmed_Data"
RESULTS_DIR="${WORKDIR}/Results"
QC_DIR="${RESULTS_DIR}/QC"
REF_GENOME="S288C_reference_genome_R64-2-1_20150113.fa"
REF_GENOME_ARCHIVE="S288C_reference_genome_Current_Release.tgz"
REF_GENOME_URL="http://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz"
 
FASTQ_R1_URL="https://ftp.ncbi.nlm.nih.gov/geo/series/GSE150nnn/GSE150800/suppl/GSE150800%5Fmm10%5FmusculusSNPStospretusSNPs.fasta.gz"
FASTQ_R2_URL="https://ftp.ncbi.nlm.nih.gov/geo/series/GSE150nnn/GSE150800/suppl/GSE150800%5Fmm10%5FmusculusSNPStospretusSNPs.fasta.gz"


CREATING DIRECTORY

In [10]:
mkdir -p "${WORKDIR}" "${REF_GENOME_DIR}" "${FASTQ_DIR}" "${TRIMMED_DIR}" "${RESULTS_DIR}" "${QC_DIR}"
cd "${WORKDIR}"


DOWNLOADING DATA

In [11]:
# Downloading the reference genome if it does not exist or is empty
if [ ! -s "${REF_GENOME_DIR}/${REF_GENOME_ARCHIVE}" ]; then
    wget -P "${REF_GENOME_DIR}" $REF_GENOME_URL
    tar -xzvf "${REF_GENOME_DIR}/${REF_GENOME_ARCHIVE}" -C "${REF_GENOME_DIR}"
else
    echo "Reference genome archive already exists and is not empty."
fi

# Downloading FASTQ files if they do not exist or are empty
if [ ! -s "${FASTQ_DIR}/$(basename ${FASTQ_R1_URL})" ]; then
    wget -P "${FASTQ_DIR}" "${FASTQ_R1_URL}"
else
    echo "FASTQ R1 file already exists and is not empty."
fi

if [ ! -s "${FASTQ_DIR}/$(basename ${FASTQ_R2_URL})" ]; then
    wget -P "${FASTQ_DIR}" "${FASTQ_R2_URL}"
else
    echo "FASTQ R2 file already exists and is not empty."
fi

--2024-01-05 10:33:54--  http://downloads.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz
Resolving downloads.yeastgenome.org (downloads.yeastgenome.org)... 52.12.181.3, 54.148.109.74, 35.81.1.134, ...
Connecting to downloads.yeastgenome.org (downloads.yeastgenome.org)|52.12.181.3|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://sgd-archive.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz [following]
--2024-01-05 10:33:55--  http://sgd-archive.yeastgenome.org/sequence/S288C_reference/genome_releases/S288C_reference_genome_Current_Release.tgz
Resolving sgd-archive.yeastgenome.org (sgd-archive.yeastgenome.org)... 52.92.236.203, 52.218.246.106, 52.92.130.59, ...
Connecting to sgd-archive.yeastgenome.org (sgd-archive.yeastgenome.org)|52.92.236.203|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21142292 (20M) [application/gzip]


QUALITY CONTROL

In [13]:
fastqc "${FASTQ_DIR}"/*.fastq.gz -o "${QC_DIR}"

Skipping '/home/annisa/Documents/teaching_linux_bash/UAS_Bioinformatics/UAS_Bioinformatics_Annisa/Fastq_Data/*.fastq.gz' which didn't exist, or couldn't be read


READ TRIMMING USING TRIMMOMATIC

In [14]:
trimmomatic PE -phred33 \
"${FASTQ_DIR}/$(basename ${FASTQ_R1_URL})" "${FASTQ_DIR}/$(basename ${FASTQ_R2_URL})" \
"${TRIMMED_DIR}/trimmed_R1.fastq.gz" "${TRIMMED_DIR}/unpaired_R1.fastq.gz" \
"${TRIMMED_DIR}/trimmed_R2.fastq.gz" "${TRIMMED_DIR}/unpaired_R2.fastq.gz" \
LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

trimmomatic: command not found


: 127

READ ALIGNMENT

In [15]:
bwa index "${REF_GENOME_DIR}/${REF_GENOME}"
bwa mem "${REF_GENOME_DIR}/${REF_GENOME}" "${TRIMMED_DIR}/trimmed_R1.fastq.gz" "${TRIMMED_DIR}/trimmed_R2.fastq.gz" > "${RESULTS_DIR}/aligned_reads.sam"

[bwa_idx_build] fail to open file '/home/annisa/Documents/teaching_linux_bash/UAS_Bioinformatics/UAS_Bioinformatics_Annisa/Reference_Genome/S288C_reference_genome_R64-2-1_20150113.fa' : No such file or directory


[E::bwa_idx_load_from_disk] fail to locate the index files


: 1

POST-ALIGNMENT PROCESSING

In [12]:
samtools view -bS "${RESULTS_DIR}/aligned_reads.sam" | samtools sort -o "${RESULTS_DIR}/sorted_aligned_reads.bam"
picard MarkDuplicates I="${RESULTS_DIR}/sorted_aligned_reads.bam" O="${RESULTS_DIR}/marked_duplicates.bam" M="${RESULTS_DIR}/marked_dup_metrics.txt"

Command 'samtools' not found, but can be installed with:
sudo apt install samtools
Command 'samtools' not found, but can be installed with:
sudo apt install samtools
Command 'picard' not found, but can be installed with:
sudo snap install picard  # version 2.10, or
sudo apt  install picard  # version 2.7.3-1
See 'snap info picard' for additional versions.


: 127

VARIANT CALLING

In [ ]:
freebayes -f "${REF_GENOME_DIR}/${REF_GENOME}" "${RESULTS_DIR}/marked_duplicates.bam" > "${RESULTS_DIR}/variants.vcf"

QUALITY CONTROL ANALYSIS WITH MULTIQC

In [ ]:
multiqc "${QC_DIR}" -o "${QC_DIR}"

GENERATING A BASIC PLOT USING R

In [ ]:
library(vcfR)
vcf <- read.vcfR('${RESULTS_DIR}/variants.vcf')

variant_summary <- summary(vcf)

pdf('${RESULTS_DIR}/variant_stats_plot.pdf')
plot(variant_summary)
dev.off()
" > "${RESULTS_DIR}/variant_stats.R"
Rscript "${RESULTS_DIR}/variant_stats.R"

echo "Workflow completed."